<a href="https://colab.research.google.com/github/Ryu4824/code-states/blob/main/n224_discussion_9%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N224 Model Tuning**

## 오늘의 목표
- 하이퍼파라미터 튜닝으로 모델의 성능을 향상시킬 수 있습니다.
- Grid/Random/Bayesian search CV를 사용할 수 있습니다.
- Grid/Random/Bayesian search CV의 차이점과 어떤 경우에 어떤 방법을 사용하면 좋을지 설명할 수 있습니다.


## **개념 Topic**
> 오늘은 하이퍼파라미터 튜닝에 대해 배웠습니다.

- grid search, randomized search, bayesian search의 차이점을 생각하면서 어떤 경우에 어떤 방법을 사용하면 좋을지 논의해 보세요.

- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

- 랜덤서치를 먼저 사용 후 추출된 값을 그리드서치로 다시 탐색하는방법이 있다

### **코치 comment**
- Grid: 시간적인 여유가 충분하고 경험적으로 해당 데이터에서 하이퍼파라미터가 어떠한 범위를 가질때 성능이 높다는 지식이 있을 경우. 정확한 탐색을 위해서 사용.
- Random: 하이퍼파라미터 범위에 대한 감이 없어서 넓은 범위로 최적의 조합을 찾으려고 하는 경우. Grid 보다 효율적으로 탐색하고자 할 때 사용.
- Bayesian: 한정된 자원 내에 Grid, Random보다 최적의 조합을 효율적으로 탐색하고자 하는 경우.
- 언제나 무조건적으로 좋은 튜닝 방법은 없기 때문에 다양한 시도를 해보고 가장 좋은 조합을 선택하는 것이 좋습니다.

scikit-learn에서 hyperopt를 구현한 라이브러리도 있습니다.
[hpsklearn 가이드 참고문서](https://hyperopt.github.io/hyperopt-sklearn/)

## **코딩 Topic**


In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 1.2 MB/s eta 0:00:00


In [ ]:
!pip install xgboost==1.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.4
    Uninstalling xgboost-1.7.4:
      Successfully uninstalled xgboost-1.7.4


### **Part.1 : 데이터 준비**
지난 노트에서 사용했던 [통신사 고객 이탈 여부 데이터](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)를 사용합니다.

> **Data Description**

- customerID : Customer ID
- gender : Whether the customer is a male or a female
- SeniorCitizen : Whether the customer is a senior citizen or not (1, 0)
- Partner : Whether the customer has a partner or not (Yes, No)
- Dependents : Whether the customer has dependents or not (Yes, No)
- tenure : Number of months the customer has stayed with the company
- PhoneService : Whether the customer has a phone service or not (Yes, No)
- MultipleLines : Whether the customer has multiple lines or not (Yes, No, No phone service)
- InternetService : Customer’s internet service provider (DSL, Fiber optic, No)
- OnlineSecurity : Whether the customer has online security or not (Yes, No, No internet service)
- OnlineBackup : Whether the customer has online backup or not (Yes, No, No internet service)
- DeviceProtection : Whether the customer has device protection or not (Yes, No, No internet service)
- TechSupport : Whether the customer has tech support or not (Yes, No, No internet service)
- StreamingTV : Whether the customer has streaming TV or not (Yes, No, No internet service)
- StreamingMovies : Whether the customer has streaming movies or not (Yes, No, No internet service)
- Contract : The contract term of the customer (Month-to-month, One year, Two year)
- PaperlessBilling : Whether the customer has paperless billing or not (Yes, No)
- PaymentMethod : The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- MonthlyCharges : The amount charged to the customer monthly
- TotalCharges : The total amount charged to the customer
- Churn : Whether the customer churned or not (Yes or No)

#### **1-1. 데이터 불러오기 및 전처리**
- 데이터셋을 불러오고 지난 노트에서 수행한 전처리 + 추가로 시도해보고 싶은 전처리를 수행해보세요.

In [ ]:
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/renewal/mldl/Customer_Churn.csv')
display(df.head())
print(df.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


(7043, 21)


In [ ]:
# !pip install pandas-profiling

In [ ]:
# from pandas_profiling import ProfileReport

# profile = ProfileReport(df, explorative=True)
# profile.to_file(output_file='df.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from category_encoders import TargetEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

target = 'Churn'
df[target] = df[target].replace({'No':0, 'Yes':1})
y = df[target]
X = df.drop(target,axis=1)

X = X.drop('customerID',axis=1)

imp = SimpleImputer(strategy='constant', fill_value='unknown')

tenc = TargetEncoder()
X_imp = pd.DataFrame(imp.fit_transform(X),columns=X.columns)
X_tenc = tenc.fit_transform(X_imp, y=y)
display(X_tenc)

enc = OrdinalEncoder()
X_enc = enc.fit_transform(X_imp)
display(X_enc)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0.269209,0.236062,0.196649,0.312791,0.619902,0.249267,0.249267,0.189591,0.417667,0.215315,0.391276,0.416355,0.335231,0.336804,0.427097,0.335651,0.452854,0.298652,0.230843
1,0.261603,0.236062,0.329580,0.312791,0.185503,0.267096,0.250442,0.189591,0.146112,0.399288,0.225021,0.416355,0.335231,0.336804,0.112695,0.163301,0.191067,0.230843,0.230843
2,0.261603,0.236062,0.329580,0.312791,0.516807,0.267096,0.250442,0.189591,0.146112,0.215315,0.391276,0.416355,0.335231,0.336804,0.427097,0.335651,0.191067,0.304783,0.360951
3,0.261603,0.236062,0.329580,0.312791,0.101083,0.249267,0.249267,0.189591,0.146112,0.399288,0.225021,0.151663,0.335231,0.336804,0.112695,0.163301,0.167098,0.230843,0.230843
4,0.269209,0.236062,0.329580,0.312791,0.516807,0.267096,0.250442,0.418928,0.417667,0.399288,0.391276,0.416355,0.335231,0.336804,0.427097,0.335651,0.452854,0.282343,0.360951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.261603,0.236062,0.196649,0.154502,0.244693,0.267096,0.286099,0.189591,0.146112,0.399288,0.225021,0.151663,0.300702,0.299414,0.112695,0.335651,0.191067,0.302636,0.230843
7039,0.269209,0.236062,0.196649,0.154502,0.016575,0.267096,0.286099,0.418928,0.417667,0.215315,0.225021,0.416355,0.300702,0.299414,0.112695,0.335651,0.152431,0.224379,0.230843
7040,0.269209,0.236062,0.196649,0.154502,0.313114,0.249267,0.249267,0.189591,0.146112,0.399288,0.391276,0.416355,0.335231,0.336804,0.427097,0.335651,0.452854,0.275868,0.230843
7041,0.261603,0.416813,0.196649,0.312791,0.471591,0.267096,0.286099,0.418928,0.417667,0.399288,0.391276,0.416355,0.335231,0.336804,0.427097,0.335651,0.191067,0.484940,0.360951


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,1,2,1,2,2,2,1,2,2,2,1,1,1,2,2,2,2,2
2,2,1,2,1,3,2,2,1,2,1,1,1,1,1,1,1,2,3,3
3,2,1,2,1,4,1,1,1,2,2,2,2,1,1,2,2,3,4,4
4,1,1,2,1,3,2,2,2,1,2,1,1,1,1,1,1,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,1,1,2,66,2,3,1,2,2,2,2,2,2,2,1,2,296,6527
7039,1,1,1,2,22,2,3,2,1,1,2,1,2,2,2,1,4,762,6528
7040,1,1,1,2,27,1,1,1,2,2,1,1,1,1,1,1,1,205,6529
7041,2,2,1,1,55,2,3,2,1,2,1,1,1,1,1,1,2,114,6530


#### **1-2. 데이터셋 분리**
- 특성과 타겟을 분리하세요.
  - 타겟은 `Churn` 입니다.
  - 타겟 데이터를 `No`는 0으로, `Yes`는 1로 변환하세요.
- 데이터를 train/test set으로 분리하세요.
  - 적절한 비율로 train/test set을 나누세요.
  - `random_state`로 시드를 고정하세요.
  - 타겟 클래스의 비율이 유지되도록 분리하세요.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tenc, y, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape,y_val.shape,y_test.shape)

(3961, 19) (1321, 19) (1761, 19)
(3961,) (1321,) (1761,)


In [ ]:
X_train

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
6205,2,1,1,1,42,2,3,2,1,1,2,1,2,1,1,1,3,766,5787
6548,2,1,1,2,37,2,2,1,1,2,2,2,1,1,1,2,2,668,6090
6743,2,1,2,1,7,2,3,3,3,3,3,3,3,3,1,1,1,1189,6266
2863,2,1,2,1,1,2,2,1,1,2,1,1,1,1,1,2,2,221,2773
232,2,1,2,1,1,2,2,2,2,2,1,1,1,2,1,1,2,202,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,1,1,2,1,61,2,2,1,1,1,1,1,1,1,1,2,1,743,1172
2449,2,1,1,2,27,2,2,1,1,2,2,1,2,1,1,1,3,1098,2379
4935,2,2,1,1,4,2,3,3,3,3,3,3,3,3,2,2,4,157,4659
3391,2,1,2,1,55,2,2,3,3,3,3,3,3,3,1,2,2,448,3263


### **Part.2 : 모델 학습 및 예측**

#### **2-1. 기준모델 설정**
- 데이터가 불균형하기 때문에 F1-score를 평가지표로 사용합니다.

  1) 타겟의 최빈값으로 기준 모델을 설정하면 F1-score가 0이 나오기 때문에 Decision Tree 모델을 기준모델로 사용합니다.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - 기준모델의 하이퍼파라미터를 자유롭게 설정하세요.

  2) `cross_val_score` 함수를 사용해서 기준모델의 검증 성능을 확인해보세요. 

  3) 검증 성능을 확인한 후 모델을 학습시켜서 학습 데이터에서의 성능을 확인해보세요. 
    - 과적합이 발생했다면 특정 하이퍼파라미터를 직접 조정하여 해소해보세요.
      - 하이퍼파라미터가 달라졌다면 다시 검증 성능을 확인한 후 학습 성능을 확인해야합니다.
    - 과적합을 해소한 기준모델의 검증 F1-score는 어떻게 되나요?
      - 이 검증 F1-score를 기준모델의 성능으로 사용합니다.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_val)
dt_cv_pred = cross_val_predict(dt_model, X_train, y_train, cv=3)
print('결정트리 평가:', classification_report(y_val, dt_pred))
print('결정트리 CV 평가:', classification_report(y_train, dt_cv_pred))

결정트리 평가:               precision    recall  f1-score   support

           0       0.82      0.81      0.81       970
           1       0.48      0.50      0.49       351

    accuracy                           0.72      1321
   macro avg       0.65      0.65      0.65      1321
weighted avg       0.73      0.72      0.73      1321

결정트리 CV 평가:               precision    recall  f1-score   support

           0       0.80      0.79      0.80      2910
           1       0.44      0.46      0.45      1051

    accuracy                           0.70      3961
   macro avg       0.62      0.62      0.62      3961
weighted avg       0.71      0.70      0.70      3961



#### **2-2. 하이퍼파라미터 튜닝**
- Grid, Random, Bayesian Search CV 중 2가지 방법을 사용해서 하이퍼파라미터 튜닝을 진행해보세요.
- 추천 튜닝 방법
  - 튜닝1) Grid/Random Search CV 중 택1
  - 튜닝2) Bayesian Search CV



##### **GridSearch CV**
- GridSearch CV를 활용해서 위의 기준모델보다 성능이 더 좋은 모델을 만들어보세요.
  - 튜닝 시, RandomForest와 XGBoost 모델 중 하나를 사용하세요.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - `random_state`로 모델의 시드를 고정하세요.
  - 튜닝 시, 하이퍼파라미터는 최소 2개, 최대 4개까지 사용하세요.
    - 시간 관계상 하이퍼파라미터 튜닝의 범위가 너무 넓지 않게 설정하세요. (튜닝 시간 3분 이내)
    - cv는 최대 3까지만 설정하세요.
    - 하이퍼파라미터의 조합과 범위를 아래와 같이 설정한 이유는 무엇인가요?
  - 최적의 하이퍼파라미터 조합을 가진 모델로 학습/검증 F1-score를 출력해보세요.
    - 검증 F1-score: `그리드서치.best_score_`
    - 최적의 모델: `그리드서치.best_estimator_`
    - [scikit-learn Gridsearch 공식문서](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
    - 기준모델보다 성능이 좋은 모델과 하이퍼파라미터 조합을 발견했나요?
      - score와 조합을 출력해보세요.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

rf_model = RandomForestClassifier(random_state=42, oob_score=True, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_val)

y_value = y.value_counts()
y_ratio = y_value[0] / y_value[1]

xgb_model = XGBClassifier(random_state=42, n_jobs=-1, booster='dart', scale_pos_weight=y_ratio)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_val)

print('랜덤트리 평가', classification_report(y_val, y_pred_rf))
print('XGB 평가', classification_report(y_val, y_pred_xgb))

랜덤트리 평가               precision    recall  f1-score   support

           0       0.83      0.90      0.86       970
           1       0.63      0.48      0.55       351

    accuracy                           0.79      1321
   macro avg       0.73      0.69      0.70      1321
weighted avg       0.78      0.79      0.78      1321

XGB 평가               precision    recall  f1-score   support

           0       0.85      0.80      0.82       970
           1       0.52      0.60      0.56       351

    accuracy                           0.75      1321
   macro avg       0.68      0.70      0.69      1321
weighted avg       0.76      0.75      0.75      1321



In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

XGB = XGBClassifier(random_state=42, n_jobs=-1, booster='dart')

# 범위 설정
params_XGB = {'learning_rate': np.linspace(0.001, 0.15, num=3),
              'colsample_bytree': np.linspace(0, 1, num=3),
              'max_depth': np.arange( 1,14, 4)}

# 목표 설정
scorer = make_scorer(f1_score, pos_label=1)

# 모델 생성
random_XGB = RandomizedSearchCV(XGB, params_XGB, cv=3, scoring=scorer)

# 학습
random_XGB.fit(X_train, y_train)

# 탐색 결과
print('최적의 파라미터', random_XGB.best_params_)
print('파라미터 적용 스코어', random_XGB.best_score_)

최적의 파라미터 {'max_depth': 1, 'learning_rate': 0.15, 'colsample_bytree': 1.0}
파라미터 적용 스코어 0.5550712809909951


##### **RandomSearch CV**
- RandomSearch CV를 활용해서 위의 기준모델보다 성능이 더 좋은 모델을 만들어보세요.
  - 튜닝 시, RandomForest와 XGBoost 모델 중 하나를 사용하세요.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - `random_state`로 모델의 시드를 고정하세요.
  - 튜닝 시, 하이퍼파라미터는 최소 2개, 최대 4개까지 사용하세요.
    - 시간 관계상 하이퍼파라미터 튜닝의 범위가 너무 넓지 않게 설정하세요. (튜닝 시간 3분 이내)
    - n_iter는 최대 10, cv는 최대 3까지만 설정하세요.
    - `random_state`로 RandomSearch CV의 시드를 고정하세요.
    - 하이퍼파라미터의 조합과 범위를 아래와 같이 설정한 이유는 무엇인가요?
  - 최적의 하이퍼파라미터 조합을 가진 모델로 학습/검증 F1-score를 출력해보세요.
    - 검증 F1-score: `랜덤서치.best_score_`
    - 최적의 모델: `랜덤서치.best_estimator_`
    - [RandomSearch 공식문서](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
    - 기준모델보다 성능이 좋은 모델과 하이퍼파라미터 조합을 발견했나요?
      - score와 조합을 출력해보세요.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

XGB = XGBClassifier(random_state=42, n_jobs=-1, booster='dart')

#범위 설정
params_XGB = ({'learning_rate': np.arange(0.001, 0.15, 0.001),
               'subsample': np.arange(0.3, 1.1, 0.1),
               'max_depth': np.arange(1, 15, 1)})
#목표설정
scorer = make_scorer(f1_score, pos_label=1)

#모델 생성
random_XGB = RandomizedSearchCV(XGB, params_XGB, cv=3, n_iter=5, scoring=scorer, random_state=42)

#학습
random_XGB.fit(X_train, y_train,eval_set=[(X_train, y_train),(X_val,y_val)], verbose=False)

#탐색결과
print('최적의 파라미터', random_XGB.best_params_)
print('파라미터 적용 스코어', random_XGB.best_score_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

rf_model = RandomForestClassifier(random_state=42, oob_score=True, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_val)

y_value = y.value_counts()
y_ratio = y_value[0] / y_value[1]

xgb_model = XGBClassifier(random_state=42, n_jobs=-1, booster='dart', scale_pos_weight=y_ratio)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_val)

print('랜덤트리 평가', classification_report(y_val, y_pred_rf))
print('XGB 평가', classification_report(y_val, y_pred_xgb))

랜덤트리 평가               precision    recall  f1-score   support

           0       0.82      0.89      0.86       970
           1       0.61      0.46      0.53       351

    accuracy                           0.78      1321
   macro avg       0.72      0.68      0.69      1321
weighted avg       0.77      0.78      0.77      1321

XGB 평가               precision    recall  f1-score   support

           0       0.85      0.81      0.83       970
           1       0.53      0.62      0.57       351

    accuracy                           0.76      1321
   macro avg       0.69      0.71      0.70      1321
weighted avg       0.77      0.76      0.76      1321



##### **BayesianSearch CV**
- BayesianSearch CV를 활용해서 위의 기준모델보다 성능이 더 좋은 모델을 만들어보세요.
  - 튜닝 시, RandomForest와 XGBoost 모델 중 하나를 사용하세요.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - `random_state`로 모델의 시드를 고정하세요.
  - 튜닝 시, 하이퍼파라미터는 최소 2개, 최대 4개까지 사용하세요.
    - 시간 관계상 하이퍼파라미터 튜닝의 범위가 너무 넓지 않게 설정하세요. (튜닝 시간 3분 이내)
    - cross_val_score
의 cv는 3 이하, fmin의 max_evals는 10 이하로 설정하세요.
    - rstate로 시드를 고정하세요.
      - `rstate = np.random.default_rng(...)`
    - 하이퍼파라미터의 조합과 범위를 아래와 같이 설정한 이유는 무엇인가요?

- BayesianSearch CV는 `best_estimator_` 메소드를 제공하고 있지 않아 최적의 조합을 찾았다면 해당 조합으로 모델을 재학습 시켜주어야합니다.
  - 발견한 하이퍼파라미터 조합을 사용해서 모델을 재학습 시키세요.
  - 재학습된 최종 모델의 학습/검증 F1-score를 출력해보세요.
  - 검증 F1-score: `-베이지안서치.best_trial["result"]["loss"]`
  - 기준모델보다 성능이 좋은 모델과 하이퍼파라미터 조합을 발견했나요?
      - score와 조합을 출력해보세요.

In [ ]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from hyperopt import hp
import numpy as np

params = {'learning_rate': hp.uniform('learning_rate', 0.001, 0.151),
          'colsample_bytree': hp.uniform('colsample_bytree', 0.8, 1),
          'max_depth': hp.randint('max_depth', 1, 12)}

def get_model(params):
    model = XGBClassifier(random_state=42, n_jobs=-1, booster='dart', scale_pos_weight=y_ratio)
    model = model.set_params(**params)
    return model


def fit_and_eval(params):
    score = cross_val_score(get_model(params), X_train, y_train, cv=3, scoring="f1")
    avg_cv_score = np.mean(score)
    return {"loss": -avg_cv_score, "status": STATUS_OK}


trials = (Trials())
best_params = fmin(fn=fit_and_eval, trials=trials, space=params, algo=tpe.suggest, max_evals=5, rstate = np.random.default_rng(2023)) 
print("최적 하이퍼파라미터: ", trials.best_trial["misc"]["vals"])
print("최적 f1: ", -trials.best_trial["result"]["loss"])

100%|██████████| 5/5 [00:24<00:00,  4.94s/trial, best loss: -0.6134962003427081]
최적 하이퍼파라미터:  {'colsample_bytree': [0.8890044832181089], 'learning_rate': [0.11566283525240263], 'max_depth': [3]}
최적 f1:  0.6134962003427081


### **Part.3 : 모델 결과 해석**

#### **3-1. 최종 모델 선정 및 일반화 성능 평가**
- 사용한 튜닝 방법 중 가장 성능이 좋은 튜닝 방법은 무엇인가요?
  - 이 방법이 **항상** 다른 튜닝 방법에 비해 성능이 좋을까요?
  - 항상 좋다면 왜 좋은지, 항상 좋진 않다면 왜 항상은 아닌지 논의해보세요.
- 가장 성능이 좋은 튜닝 방법을 사용한 모델을 최종 모델로 선정하세요.
  - 최종 모델의 일반화 성능(test set에서의 F1-score)을 확인해보세요.
  - 과적합이 발생하진 않았나요?
    - 과적합이 발생했다면 어떻게 해소할 수 있을지 논의해보세요.

In [ ]:
model = XGBClassifier(random_state=42, n_jobs=-1, booster='dart', scale_pos_weight=y_ratio,
                      colsample_bytree= 0.8890044832181089, learning_rate= 0.11566283525240263, max_depth=3)
model.fit(X_train,y_train)
y_pred = model.predict(X_val)

print('XGB 평가', classification_report(y_val, y_pred))

XGB 평가               precision    recall  f1-score   support

           0       0.92      0.73      0.82       970
           1       0.53      0.83      0.64       351

    accuracy                           0.76      1321
   macro avg       0.72      0.78      0.73      1321
weighted avg       0.82      0.76      0.77      1321



In [ ]:
model = XGBClassifier(random_state=42, n_jobs=-1, booster='dart', scale_pos_weight=y_ratio,
                      colsample_bytree= 0.8890044832181089, learning_rate= 0.11566283525240263, max_depth=3)
model.fit(X_train,y_train)
y_pred_test = model.predict(X_test)

print('XGB 평가', classification_report(y_test, y_pred_test))

XGB 평가               precision    recall  f1-score   support

           0       1.00      0.98      0.99      1294
           1       0.95      0.99      0.97       467

    accuracy                           0.98      1761
   macro avg       0.97      0.98      0.98      1761
weighted avg       0.98      0.98      0.98      1761



### **Conclusion**
> 오늘 Topic을 수행한 결과를 바탕으로, 다음 사항에 대해 답해 주세요.

1. 어떤 튜닝 방법들을 사용했나요?
2. 튜닝에 사용한 하이퍼파라미터 조합과 범위의 설정 이유는 무엇인가요?
3. 가장 좋은 튜닝 방법과 해당 모델의 학습/검증/테스트 성능은 어떻게 되나요?
4. 일반화 성능(test set 성능)을 올리기 위해서 어떤 방법을 사용해볼 수 있을까요?

- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

1. 랜덤CV/그리드CV/베이지안
2. max_depth, learning_rate, colsample_bytree를 사용했습니다. 가장 영향이 많이간다고 생각했습니다
3. 베이지안 / 타겟인코딩 데이터 / XGB 사용했을때의 test f1스코어 0.97
4. EDA와 특성공학, 하이퍼파라미터의 최적값 탐색

## **심화 Topic(Optional)**
- 위의 **2-2. 하이퍼파라미터 튜닝** 파트에서 사용하지 않은 나머지 튜닝 방법을 주어진 가이드에 따라 수행해보세요.

In [ ]:
XGB = XGBClassifier(random_state=42, n_jobs=-1, booster='dart')

# 범위 설정
params_XGB = {'learning_rate': np.arange(0.05, 0.12,0.05 ),
              'colsample_bytree': np.arange(0.75, 1,0.05 ),
              'max_depth': np.arange( 1,6, 1)}

# 목표 설정
scorer = make_scorer(f1_score, pos_label=1)

# 모델 생성
grid_XGB = GridSearchCV(XGB, params_XGB, cv=3, scoring=scorer)

# 학습
grid_XGB.fit(X_train, y_train)

# 탐색 결과
print('최적의 파라미터', grid_XGB.best_params_)
print('파라미터 적용 스코어', grid_XGB.best_score_)
#너무 오래걸려요...

최적의 파라미터 {'colsample_bytree': 0.9000000000000001, 'learning_rate': 0.1, 'max_depth': 2}
파라미터 적용 스코어 0.965743263271896
